In [62]:
from pathlib import Path
ciphertext = Path("./ciphertext").read_text()
print(ciphertext)

E0ECE8BFB0C9854BC9916246DC1E7EC42994C78EBC0796690E7E0385FA49EA367CD829
E046538A205A27B6848E26C274FD1494A930F64E0E7BE70DDCEC6DB9CAED505D4E8F77
5E4AB8920E02B1010869A96EBBB65B6BA6D78A733735A0D890D6AF11586CB504FDCAD9
8CB1D1BAF7DA4A0F205304D1F7596AE23E9414FD2B56458CC1961C131C52524BF7B2A1
5140E943D61AA53F280340693612F8A9551D2406CE6CF66FCAB6F925BD5EB76CFB25945
740D229F0D125E6DADDFA1FACA411E93AE56DFD27F186F30DB22BC79C17594F16FE414
57D2C769EF08201B0FF91D482BF92EAA0AEE4991009C8717EFB6DC0CD0B535E38EB13E
E4AC65FCE00E82C6587FECBC9EC550DDB66587D5735B1DB78BFB8AF54F1F237D2A2EE
AB2B61D195105CBB6557644B2474ED96DBB918DE09D0B17DED901BE61C97A1CD3B200
A3678369FF4



In [63]:
#16 bits to be broken down into 4 different words per key
#There are 4 bits in 1 hex digit, se what we need to do is divide the key
#into 4 different 
#https://linuxhint.com/string-to-hexadecimal-in-python/
#https://stackoverflow.com/questions/2397687/whats-the-easiest-way-to-convert-a-list-of-hex-byte-strings-to-a-list-of-hex-in

key11 = "E593B292_D82CE2C4_8539C08C_2EFB24EF"#k11 is the 10th key, we start at k1
key11array = key11.split("_")
key11array = [int(x, 16) for x in key11array]
#for i in range(len(key11array)):
 #   key11array[i] = hex(key11array[i])
for i in key11array:
    print(hex(i))
#key10 = F6FA49F0 3DBF5056 5D152248 ABC2E463

0xe593b292
0xd82ce2c4
0x8539c08c
0x2efb24ef


In [78]:
#Here, do the XOR operations
#
val4 = key11array[3] ^ key11array[2]#abc2e463, do G of this value
val3 = key11array[2] ^ key11array[1]
val2 = key11array[1] ^ key11array[0]
#for val1 we need to XOR G(array[3]) with val1
#So essentially take G(the last element) and XOR is with the first element to get val1
#We need to shift to the right 8 places to shift everything over 1 byte
#Got this code snippet from https://www.geeksforgeeks.org/rotate-bits-of-an-integer/
num1 = hex(val4)
newarray = [None] * len(num1)
for i in range(len(num1)):
    newarray[i] = num1[i + 2 + i : i + 4 + i]
newarray = newarray[0 : 4]
temparray = [None] * len(num1)
temparray[0] = newarray[1]
temparray[1] = newarray[2]
temparray[2] = newarray[3]
temparray[3] = newarray[0]
temparray = temparray[0 : 4]

    
#Just do it the hard way Matt
print(hex(key11array[3]))
print(num1)
print(newarray)
print(temparray)

0x2efb24ef
0xabc2e463
['ab', 'c2', 'e4', '63']
['c2', 'e4', '63', 'ab']


In [48]:
#Now do rc of the first byte
#https://en.wikipedia.org/wiki/AES_key_schedule#Rcon
#RC coefficient per round in hexadecimal: 01, 02, 04, 08, 10, 20, 40, 80, 1B, 36
string = num1
temp = string
string2 = string[2: 4]
num2 = int(string2, 16) ^ int("01", 16)#first byte in num1
hex(num2)#now replace this with the first byte in num1
string = string[:2] + hex(num2) + string[4:]
#s = s[:index] + newstring + s[index + 1:]
string = string[2:]
print(temp)
print(string)#Now XOR this with the first element:
val1 = int(string, 16) ^ key11array[0]
hex(val1)
#key10 = F6FA49F0 3DBF5056 5D152248 ABC2E463
print(hex(val4))
print(hex(val3))
print(hex(val2))
print(hex(val1))

0x2efb24ef
0x2ffb24ef
0xabc2e463
0x5d152248
0x3dbf5056
0xca68967d


In [79]:
#https://www.reddit.com/r/learnprogramming/comments/syb7wk/python_rijndael_sbox_in_hex/
sBox = [0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
        0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
        0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
        0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
        0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
        0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
        0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
        0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
        0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
        0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
        0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
        0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
        0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
        0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
        0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
        0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16]
#16*12 + 2#Formula is first hex value * 16 + second hex value
#(0xF + 1) * 0xC + 0x2# The hexadecimal way
#hex(sBox[194])

In [80]:
#0xef2efb24, after shift, start here

#0xabc2e463
#0x5d152248
#0x3dbf5056

#key10 = F6FA49F0 3DBF5056 5D152248 ABC2E463

#Goal, do a sub on each byte of the shifted 0xef2efb24
#string = hex(num1)
#q = 0
#myarray = [None] * len(string)
#print(len(myarray))
#https://stackoverflow.com/questions/10712002/create-an-empty-list-with-certain-size-in-python
#for i in range(len(string)):
    #myarray[q] = string[q + 2 + q : q + 4 + q]
    #q+=1
    
#myarray = myarray[0 : 4]
#myarray
for i in temparray:
    print(i)

c2
e4
63
ab


In [81]:
#Now do the byte substitution: 
#sBox is the name of the sbyte array
for i in range(len(temparray)):#do the think
    first = temparray[i][0 : 1]
    second = temparray[i][1]
    temparray[i] = sBox[int(first, 16) * 16 + int(second, 16)]
for i in temparray:
    print(hex(i))
    

0x25
0x69
0xfb
0x62


In [83]:
myval = 0xdf31f36#This is after byte substitution
newval = int("25", 16) ^ int("36", 16)
hex(newval)

'0x13'

In [84]:
myval = int("1369fb62", 16) ^ key11array[0]
print(hex(myval))
print(hex(val2))
print(hex(val3))
print(hex(val4))
#F6FA49F0
#We should get 1369fb62

0xf6fa49f0
0x3dbf5056
0x5d152248
0xabc2e463
